! pip install azure-search-documents --pre
! pip install openai
! pip install python-dotenv
! pip install --upgrade azure-search-documents
!pip install --upgrade langchain
! pip install pandasql

In [8]:
# Import required libraries  
import os  
import json  
import openai  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SemanticSettings,  
    VectorSearch, 
    HnswVectorSearchAlgorithmConfiguration,  
)  

from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import CharacterTextSplitter
from azure.storage.blob import BlobServiceClient
import tempfile
from langchain.document_loaders import PyPDFLoader
from azure.core.exceptions import HttpResponseError 
from langchain.embeddings.openai import OpenAIEmbeddings

In [9]:
# Configure environment variables  
#load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
credential = AzureKeyCredential("jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg")

In [10]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://testingchat.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "e8143eabb02541259054426630db12a7"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
model: str = "text-embedding-ada-002"
#pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"     

AZURE_SEARCH_SERVICE_ENDPOINT = "https://genaisafety.search.windows.net"
AZURE_SEARCH_ADMIN_KEY = "jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg"

In [11]:
def create_index(AZURE_SEARCH_INDEX_NAME, fields):
    vector_search = VectorSearch(
        algorithm_configurations=[
            HnswVectorSearchAlgorithmConfiguration(
                name="my-vector-config",
                kind="hnsw",
                parameters={
                    "m": 4,
                    "efConstruction": 400,
                    "efSearch": 500,
                    "metric": "cosine"
                }
            )
        ]
    )


    semantic_config = SemanticConfiguration(
        name="my-semantic-config",
        prioritized_fields=PrioritizedFields(
            title_field=SemanticField(field_name="id"),
            prioritized_keywords_fields=[SemanticField(field_name="content")]
        )
    )

    # Create the semantic settings with the configuration
    semantic_settings = SemanticSettings(configurations=[semantic_config])

    # Create the search index with the semantic settings
    index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME, fields=fields,
                        vector_search=vector_search, semantic_settings=semantic_settings)

    result = search_client.create_or_update_index(index)
    return result

In [12]:
embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 

# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Replace with your container name
container_name = "safety"

# Create a BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

In [6]:
blobs = container_client.list_blobs()
dict_data = []
for blob in blobs:
    if blob.name.startswith("etq-data"):
        #blob_client = container_client.get_blob_client(blob)
        pdf_file_name=str(blob.name).split("/")[1] 
        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        print(bid)

vw-etq-events-fast
vw-reportit-allreport-fast


In [7]:
import pandas as pd
import io
from azure.storage.blob import BlobServiceClient

# Replace with your actual connection string
azure_connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Replace with your container name
container_name = "safety"

try:
    # Connect to Azure Blob Storage
    blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
    container_client = blob_service_client.get_container_client(container_name)

    # List all blobs in the container
    blobs = container_client.list_blobs()

    # Create an empty list to store DataFrame objects
    dfs = {}

    # Loop through the blobs and process those that match the prefix
    for blob in blobs:
        if blob.name.startswith("etq-data"):
            # Extract file name and bid name
            file_name = str(blob.name).split("/")[1]
            bid_name = str(blob.name).split("/")[1].split(".")[0].lower().replace(" ", "_").replace("_", "-")

            # Get the blob client for reading the blob data
            blob_client = container_client.get_blob_client(blob.name)
            print(blob.name)

            # Download the blob data as a stream
            blob_data = blob_client.download_blob()

            # Read the stream as a string and then into a Pandas DataFrame
            blob_text = blob_data.readall().decode('utf-8')
            df = pd.read_csv(io.StringIO(blob_text))

            # Append the DataFrame to the list
            dfs[file_name] = df

    # Now you can work with the list of DataFrames (dfs) as needed
    print(dfs.keys())

except Exception as e:
    print(f'Error: {str(e)}')


etq-data/vw-ETQ-Events-fast
etq-data/vw_ReportIt_AllReport_fast
dict_keys(['vw-ETQ-Events-fast', 'vw_ReportIt_AllReport_fast'])


In [8]:
dfs['vw-ETQ-Events-fast'].head(1)

,Division,INCIDENT_ID,EtqNumber,EventDate,Airline,Flight,Tail,Risk,ReportType,IrropType,...,RiskNotes,CreatedDate,Likelihood,Severity,ErrStation,DiscStation,EventLevel,DAGSRB,InvestReq,Time
0,Inflight,205,204,2016-10-28,NaN,2094,428QX,1,Inflight : Other Event,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN


In [9]:
dfs['vw_ReportIt_AllReport_fast'].head(1)

,Carrier,EventType,SubType,Division,ID,ObjectId,WeekStart,SubmittedDate,Title,Risk,...,Description,Category,EventDate,Recommendations,Linked_eventNumber,Source,ErrorStation,DiscoveredStation,PSID,DISPLAY_NAME
0,AS,Report,Irreg,Inflight,49020,49051,2018-01-01,2018-01-01 00:29:25,Inflight : Catering/Service Interruption,NaN,...,Was only catered with one beverage cart on a d...,Meal Issue,2017-12-31 12:00:00,NaN,39303.0,Mobile Application,NaN,NaN,1050875,Alexandra Prewett


In [10]:
len(dfs['vw-ETQ-Events-fast']['EtqNumber'].unique())

296968

In [11]:
len(dfs['vw_ReportIt_AllReport_fast']['Linked_eventNumber'].unique())

295493

In [ ]:
# Find elements that are in array1 but not in array2
#not_in_array2 = list(set(dfs['vw-ETQ-Events-fast']['EtqNumber'].unique()) - set(dfs['vw_ReportIt_AllReport_fast']['Linked_eventNumber'].unique()))

# Find elements that are in array2 but not in array1
#not_in_array1 = list(set(dfs['vw_ReportIt_AllReport_fast']['Linked_eventNumber'].unique()) - set(dfs['vw-ETQ-Events-fast']['EtqNumber'].unique()))

#print("Elements in array1 but not in array2:", not_in_array2)
#print("Elements in array2 but not in array1:", not_in_array1)

In [12]:
# 1. Perform a left join between etq_df and ri_df
merged_df = pd.merge(dfs['vw-ETQ-Events-fast'], dfs['vw_ReportIt_AllReport_fast'], left_on='EtqNumber', right_on='Linked_eventNumber', how='left')
merged_df.head(1)

,Division_x,INCIDENT_ID,EtqNumber,EventDate_x,Airline,Flight_x,Tail_x,Risk_x,ReportType,IrropType,...,Description_y,Category,EventDate_y,Recommendations,Linked_eventNumber,Source,ErrorStation,DiscoveredStation,PSID,DISPLAY_NAME
0,Inflight,205,204,2016-10-28,NaN,2094,428QX,1,Inflight : Other Event,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
merged_df.shape

(356440, 53)

In [14]:
merged_df.columns

Index(['Division_x', 'INCIDENT_ID', 'EtqNumber', 'EventDate_x', 'Airline',
       'Flight_x', 'Tail_x', 'Risk_x', 'ReportType', 'IrropType', 'Cause',
       'CauseOther', 'Subject', 'Description_x', 'Origin', 'Destination',
       'Location_x', 'WSRDate', 'OPLEOI', 'AnalystNotes', 'RiskNotes',
       'CreatedDate', 'Likelihood', 'Severity', 'ErrStation', 'DiscStation',
       'EventLevel', 'DAGSRB', 'InvestReq', 'Time', 'Carrier', 'EventType',
       'SubType', 'Division_y', 'ID', 'ObjectId', 'WeekStart', 'SubmittedDate',
       'Title', 'Risk_y', 'Location_y', 'Tail_y', 'Flight_y', 'Description_y',
       'Category', 'EventDate_y', 'Recommendations', 'Linked_eventNumber',
       'Source', 'ErrorStation', 'DiscoveredStation', 'PSID', 'DISPLAY_NAME'],
      dtype='object')

In [15]:
# 2. Apply the filtering conditions
event_start_date = '2023-08-01'
event_end_date = '2023-09-15'

filtered_df = merged_df[(merged_df['EventDate_x'] >= event_start_date) &
                        (merged_df['EventDate_x'] < event_end_date) &
                        (merged_df['Recommendations'].notnull())]

# 3. Select the desired columns
selected_columns = ['ID','Division_x', 'EventDate_x', 'Subject', 'Description_x', 'Recommendations']
result_df = filtered_df[selected_columns]
result_df.head(1)

,ID,Division_x,EventDate_x,Subject,Description_x,Recommendations
8,453873.0,Inflight,2023-09-01,Inflight service cxld due to turb,- Report # 453873\n \nSuspended service due to...,Safety is the most important


In [16]:
selected_columns = ['ID','Division_x', 'EventDate_x', 'Subject', 'Description_x', 'Recommendations']
result_df = filtered_df[selected_columns] #filtered rows
df_concat = result_df.copy()
df_concat.head(2)

,ID,Division_x,EventDate_x,Subject,Description_x,Recommendations
8,453873.0,Inflight,2023-09-01,Inflight service cxld due to turb,- Report # 453873\n \nSuspended service due to...,Safety is the most important
76,451154.0,AOCS,2023-08-22,AUGUST (2) - IB Medical event at arrival stati...,- Report # 451659\n \nPAX EXPIRENCED MEDICAL I...,Required


In [17]:
df_concat.shape

(3620, 6)

In [18]:
df_concat = df_concat[df_concat['ID'].notna()] #handling null IDs
df_concat.shape

(3620, 6)

In [19]:
# Convert all columns to string type
df_concat = df_concat.astype(str)

# Concatenate all columns' data into a single column
df_concat['Concatenated'] = df_concat[["Subject","Description_x","Recommendations"]].apply(lambda row: ' '.join(row), axis=1)

In [20]:
# Embed the 'Concatenated' column
df_concat['content_vector'] = df_concat['Concatenated'].apply(lambda text: embeddings.embed_query(str(text)))

In [28]:
df_concat['id_vector'] = df_concat['ID'].apply(lambda text: embeddings.embed_query(str(text)))

In [29]:
df_concat['division_vector'] = df_concat['Division_x'].apply(lambda text: embeddings.embed_query(str(text)))

In [30]:
df_concat['eventDate_vector'] = df_concat['EventDate_x'].apply(lambda text: embeddings.embed_query(str(text)))

In [31]:
df_concat['subject_vector'] = df_concat['Subject'].apply(lambda text: embeddings.embed_query(str(text)))

In [32]:
df_concat['description_vector'] = df_concat['Description_x'].apply(lambda text: embeddings.embed_query(str(text)))

In [33]:
df_cocat['recommendations_vector'] = df_concat['Recommendations'].apply(lambda text: embeddings.embed_query(str(text)))

In [75]:
final_doc = []
metadata = df_concat.columns.tolist()
for index, row in df_concat.iterrows():
    final_doc.append({'id': str(int(float(df_concat['ID'][index]))),
                      'content': df_concat['Concatenated'][index],
                      'content_vector': df_concat['content_vector'][index],
                      'subject':df_concat['Subject'][index],
                      'subject_vector': df_concat['subject_vector'][index],
                      'metadata': str(metadata),
                     })

In [76]:
len(final_doc)

3620

In [77]:
# Display the resulting DataFrame
df_concat.head(2)

,ID,Division_x,EventDate_x,Subject,Description_x,Recommendations,Concatenated,content_vector,id_vector,division_vector,eventDate_vector,subject_vector,description_vector,recommendations_vector
8,453873.0,Inflight,2023-09-01,Inflight service cxld due to turb,- Report # 453873\n \nSuspended service due to...,Safety is the most important,Inflight service cxld due to turb - Report # 4...,"[-0.011356354578211858, -0.00609373667739792, ...","[-0.016248947079701224, -0.007459161835378162,...","[-0.0059331738186065835, -0.007777998685523574...","[-0.00939791308327018, -0.011207281053513048, ...","[-0.023884910534913276, -0.019561036966515823,...","[-0.01992973646531766, -0.024288931389472003, ...","[0.011368890765283026, -0.0021760767844222406,..."
76,451154.0,AOCS,2023-08-22,AUGUST (2) - IB Medical event at arrival stati...,- Report # 451659\n \nPAX EXPIRENCED MEDICAL I...,Required,AUGUST (2) - IB Medical event at arrival stati...,"[-0.0074294572127883846, 0.011226313107791583,...","[-0.00807976643502035, 0.01599579704537728, -0...","[-0.010615121672028362, -0.006976238982502792,...","[-0.017820404347295527, -0.013239759583918275,...","[-0.019318638354735372, -0.001148878828679178,...","[0.004142933175630319, 0.016348543676483515, 0...","[-7.77843225158927e-05, -0.006970893515425104,..."


In [56]:
--reference
# Determine the total number of rows in the DataFrame
total_rows = len(df_concat)

# Calculate the number of rows in each half
rows_per_half = total_rows // 200

In [ ]:
--reference
# Create slices of the DataFrame for each half
dataframes_list = []
data_dict_list = []
partitionid = []
#ID = []
Content = []
Content_Vector = []
for i in range(200):
    start_row = i * rows_per_half
    end_row = (i + 1) * rows_per_half
    
    df_slice = df_concat.iloc[start_row:end_row] #slice df_concat
    #dataframes_list.append(df_slice)
    #convert DF to JSON
    partitionid.append(i)
    for index in df_slice['ID'].index:
        Content.append(df_slice['Concatenated'][index])
        Content_Vector.append(df_slice['content_vector'][index])
        
        ID.append(df_slice['ID'][index])
        id_vector.append(df_slice['id_vector'][index])
        
        Division.append(df_slice['Division_x'][index])
        division_vector.append(df_slice['division_vector'][index])
        
        EventDate.append(df_slice['EventDate_x'][index])
        eventDate_vector.append(df_slice['eventDate_vector'][index])
        
        Subject.append(df_slice['Subject'][index])
        subject_vector.append(df_slice['subject_vector'][index])
        
        Description.append(df_slice['Description_x'][index])
        description_vector.append(df_slice['description_vector'][index])
        
        Recommendations.append(df_slice['Recommendations'][index])
        recommendations_vector.append(df_slice['recommendations_vector'][index])
        
    json_data = df_slice.to_json()
    data_dict = json.loads(json_data) #200
    data_dict_list.append(data_dict)

In [ ]:
--creating new vectors by modifying embedded vectors

In [80]:
AZURE_SEARCH_INDEX_NAME=bid_name + "-subject-vector-index"
print(AZURE_SEARCH_INDEX_NAME)

search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)


vw-reportit-allreport-fast-subject-vector-index


In [81]:
AZURE_SEARCH_INDEX_NAME 

'vw-reportit-allreport-fast-subject-vector-index'

In [82]:
try:
    existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
except Exception as e:
    existing_index = None
    # The index doesn't exist, so create a new one
    
metadata = result_df.columns.tolist()
if existing_index == None:
    fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
    SearchField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchableField(name="subject", type=SearchFieldDataType.String, searchable=True),
    SearchField(name="subject_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),
    ]  

    
    result = create_index(AZURE_SEARCH_INDEX_NAME, fields)
    print(f' {result.name} created index') 
    
    
    ######  

    for batch in final_doc:
        try:
            result = search_client_v2.upload_documents(batch)
            print('documents uploaded')
        except Exception as e:
            print(f"Error uploading documents: {e}")
            print(repr(e))
            break

    print("Vector store added successfully")

    ######
                       

else:
    try:
        # Check if the index exists
        # existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
        # print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")
        ######
        for batch in final_doc:
            try:
                result = search_client_v2.upload_documents(batch)
                print('documents uploaded')
            except Exception as e:
                print(f"Error uploading documents: {e}")
                print(repr(e))
                break
        print("Vector store added successfully")

    except Exception as e:
        print(e)
        pass

 vw-reportit-allreport-fast-subject-vector-index created index
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
docume

documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents up

documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents up

documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents up

documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents up

VECTOR SEARCH SIMILARITY - multiple fields

In [101]:
print(AZURE_SEARCH_INDEX_NAME)

search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)

vw-reportit-allreport-fast-subject-vector-index


In [92]:
# Pure Vector Search
query = "show me pet related data"

embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 

query_embeddings = embeddings.embed_query(query)

vector = Vector(value=query_embeddings, k=1, fields="content_vector, subject_vector")

  
results = search_client_v2.search(  
    search_text=None,  
    vectors= [vector],
    select=["id" ,"content", "metadata"],
)  

In [93]:
for result in results:  
    print(f"id: {result['id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"content: {result['content']}") 
    print(f"metadata: {result['metadata']}\n") 
    break

id: 455788
Score: 0.03333333507180214
content: Pet not in crate - Report # 455788
 
Pet in cabin not in a crate. She had the dog in a cross body sling. She did not tell us she has a medical email approval from us. We finally got her to put the dog under the seat and to leash it. She offered no details of it being a service animal just a pet Maybe it would be helpful to have an easy accessible pop up about service animal rules.
metadata: ['ID', 'Division_x', 'EventDate_x', 'Subject', 'Description_x', 'Recommendations', 'Concatenated', 'content_vector', 'id_vector', 'division_vector', 'eventDate_vector', 'subject_vector', 'description_vector', 'recommendations_vector']



VECTOR SEARCH SIMILARITY - multiple vectors

In [94]:
# Pure Vector Search
query = "show me pet related data"

embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 

query_embeddings = embeddings.embed_query(query)

vector1 = Vector(value=query_embeddings, k=1, fields="content_vector")
vector2 = Vector(value=query_embeddings, k=1, fields="subject_vector")
  
results = search_client_v2.search(  
    search_text=None,  
    vectors= [vector1, vector2],
    select=["id" ,"content", "metadata"],
)  

In [95]:
for result in results:  
    print(f"id: {result['id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"content: {result['content']}") 
    print(f"metadata: {result['metadata']}\n") 
    break

id: 455788
Score: 0.03333333507180214
content: Pet not in crate - Report # 455788
 
Pet in cabin not in a crate. She had the dog in a cross body sling. She did not tell us she has a medical email approval from us. We finally got her to put the dog under the seat and to leash it. She offered no details of it being a service animal just a pet Maybe it would be helpful to have an easy accessible pop up about service animal rules.
metadata: ['ID', 'Division_x', 'EventDate_x', 'Subject', 'Description_x', 'Recommendations', 'Concatenated', 'content_vector', 'id_vector', 'division_vector', 'eventDate_vector', 'subject_vector', 'description_vector', 'recommendations_vector']



VECTOR SEARCH SIMILARITY - multiple indexes

In [1]:
AZURE_SEARCH_INDEX_NAME1 = "aag-safety-manual-index"
AZURE_SEARCH_INDEX_NAME2 = "vw-reportit-allreport-fast-subject-vector-index"

query = "show me pet related data"

In [6]:
def vector_search_similarity(SEARCH_INDEX_NAME,query):
    search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=SEARCH_INDEX_NAME, credential=credential)

    embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 

    query_embeddings = embeddings.embed_query(query)

    vector1 = Vector(value=query_embeddings, k=1, fields="content_vector")
    #vector2 = Vector(value=query_embeddings, k=1, fields="subject_vector")

    results = search_client_v2.search(  
        search_text=None,  
        vectors= [vector1],
        select=["id" ,"content", "metadata"],
    )  
    return results


In [19]:
search_results = vector_search_similarity(AZURE_SEARCH_INDEX_NAME2,query)

In [20]:
for result in search_results:  
    print(f"id: {result['id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"content: {result['content']}") 
    print(f"metadata: {result['metadata']}\n") 
    break

id: 455788
Score: 0.8337135
content: Pet not in crate - Report # 455788
 
Pet in cabin not in a crate. She had the dog in a cross body sling. She did not tell us she has a medical email approval from us. We finally got her to put the dog under the seat and to leash it. She offered no details of it being a service animal just a pet Maybe it would be helpful to have an easy accessible pop up about service animal rules.
metadata: ['ID', 'Division_x', 'EventDate_x', 'Subject', 'Description_x', 'Recommendations', 'Concatenated', 'content_vector', 'id_vector', 'division_vector', 'eventDate_vector', 'subject_vector', 'description_vector', 'recommendations_vector']



NEW WORK

In [ ]:
try:
    existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
except Exception as e:
    existing_index = None
    # The index doesn't exist, so create a new one
    
metadata = result_df.columns.tolist()
if existing_index == None:
    fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="Division", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="EventDate", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="subject", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="description", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="Recommendations", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),
    ]

    result = create_index(AZURE_SEARCH_INDEX_NAME, fields)
    print(f' {result.name} created index')                        
    final_doc=[]
    for index in range(len(partitionid)):
        final_doc.append({'id': str(partitionid[index]), 
                          'Division': Division[index],
                          'EventDate': EventDate[index],  #df.iloc[index],
                          'subject': Subject[index],
                          'description': Description[index],
                          'Recommendations': Recommendations[index],
                          'content': str(data_dict_list[index]),
                          'metadata': str(metadata),
                         })

     
    # Define a list to store batches of documents
    #document_batches = [final_doc[i] for i in range(0, len(final_doc))]

    # Upload documents in batches
    for batch in final_doc:
        try:
            result = search_client_v2.upload_documents(batch)
            print('documents uploaded')
        except Exception as e:
            print(f"Error uploading documents: {e}")
            print(repr(e))
            break

    print("Vector store added successfully")

else:
    try:
        # Check if the index exists
        # existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
        # print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")
        final_doc=[]
        for index in range(len(partitionid)):
            final_doc.append({'id': str(partitionid[index]), 
                              'Division': Division[index],
                              'EventDate': EventDate[index],  #df.iloc[index],
                              'subject': Subject[index],
                              'description': Description[index],
                              'Recommendations': Recommendations[index],
                              'content': str(data_dict_list[index]),
                              'metadata': str(metadata),
                             })

        

        # Define a list to store batches of documents
        document_batches = [final_doc[i] for i in range(0, len(final_doc))]

        # Upload documents in batches
        for batch in document_batches:
            try:
                result = search_client_v2.upload_documents(batch)
                print('documents uploaded')
            except Exception as e:
                print(f"Error uploading documents: {e}") 
                print(repr(e))
                break

        print("Vector store added successfully")


    except Exception as e:
        print(e)
        pass